###  Importing Wandb and Login using Key


In [ ]:
%%capture
!pip install wandb
import wandb
wandb.login(key ="9c0cf96a5b886197f51883781f17b735b2ac32b1")

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')
# !wget 'https://drive.google.com/uc?export=download&id=1uRKU4as2NlS9i8sdLRS1e326vQRdhvfw' -O dataset.zip && unzip dataset.zip

## Importing Libraries

In [ ]:
import plotly.graph_objs as go
from __future__ import unicode_literals, print_function, division
from io import open
import unicodedata
import string
import re
import random
import numpy as np
from torch.utils.data import TensorDataset, DataLoader
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
import torch.nn.functional as F

## Initialize Seed

In [ ]:
#-----------------------Initialize Device---------------------------

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.empty_cache()
print(device)


#---------------------Start Seed------------------------------------

import os
def seed_everything(seed=1):
    os.environ["PYTHONHASHSEED"] = str(seed)
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False


seed_everything()

## Encode Data

In [ ]:

class Lang:
    def __init__(self, name):
        self.name = name
        self.char2index = {'#': 0, '$': 1, '^': 2}
        self.char2count = {'#': 1, '$': 1, '^': 1}
        self.index2char = {0: '#', 1: '$', 2: '^'}
        self.n_chars = 3  # Count
        self.data = {}
        

    def addWord(self, word):
        for char in word:
            self.addChar(char)

    def addChar(self, char):
        if char not in self.char2index:
            self.char2index[char] = self.n_chars
            self.char2count[char] = 1
            self.index2char[self.n_chars] = char
            self.n_chars += 1
        else:
            self.char2count[char] += 1




class EncodedData:
  def __init__(self,lang):
    
    #------------------------------------For Kaggle interFace---------------------------------

    self.train_df = pd.read_csv(f"/kaggle/input/dlassgn/hin_train.csv", header = None)
    self.val_df = pd.read_csv(f"/kaggle/input/dlassgn/hin_valid.csv", header = None)
    self.test_df = pd.read_csv(f"/kaggle/input/dlassgn/hin_test.csv", header = None)

    #------------------------------------For colab interface-------------------------------------

#     self.train_df = pd.read_csv(f"drive/MyDrive/aksharantar_sampled/{lang}/{lang}_train.csv", header = None)
#     self.val_df = pd.read_csv(f"drive/MyDrive/aksharantar_sampled/{lang}/{lang}_valid.csv", header = None)
#     self.test_df = pd.read_csv(f"drive/MyDrive/aksharantar_sampled/{lang}/{lang}_test.csv", header = None)


    self.input_lang = Lang('eng')
    self.output_lang = Lang(lang)

    # add the words to the respective languages
    for i in range(len(self.train_df)):
        
        self.input_lang.addWord(self.train_df[0][i])
        self.output_lang.addWord(self.train_df[1][i])
    
    max_len_1,max_len_2 = self.max_word_lengths(self.train_df)
    self.train_xE,self.train_yE = self.preprocessing(self.train_df,max_len_1,max_len_2)

    max_len_1,max_len_2 = self.max_word_lengths(self.val_df)
    self.val_xE,self.val_yE = self.preprocessing(self.val_df,max_len_1,max_len_2)

    max_len_1,max_len_2 = self.max_word_lengths(self.test_df)
    self.test_xE,self.test_yE = self.preprocessing(self.test_df,max_len_1,max_len_2)

    
  #-------------------Calculate max length of word in input and output----------

  def max_word_lengths(self,df):
    max_len_1 = df.iloc[:,0].str.len().max()
    max_len_2 = df.iloc[:,1].str.len().max()
    return max_len_1, max_len_2


  #-----------------Preprocess on input and output------------------------------

  def preprocessing(self,df,max_len_1,max_len_2):
    
    M1 = np.zeros((len(df),max_len_1+1))
    M2 = np.zeros((len(df),max_len_2+2))

    
    
    # ---------------------------Encode column 1-------------------------------

    col1 = df.iloc[:, 0].str.lower().str.split()

    for i in range(len(col1)) :
      word = col1[i][0]
      # print(word," " ,word[0],word[1])
      for j in range(len(word)):
        char =word[j]
        if(self.input_lang.char2index.get(char) is not None):
          M1[i][j]=self.input_lang.char2index.get(char)
        else:
          M1[i][j]=1
    
    # -----------------------Encode column 2-----------------------------------

    col2 = df.iloc[:, 1].str.lower().str.split()

    for i in range(len(col2)) :


      word = col2[i][0]
  
      M2[i][0]=2
      for j in range(len(word)):
        char =word[j]
        if(self.output_lang.char2index.get(char) is not None):
          M2[i][j+1]=self.output_lang.char2index.get(char)
        else:
          M2[i][j+1]=1

    
    return torch.from_numpy(M1),torch.from_numpy(M2)

## EncoderRNN DecoderRNN 

In [ ]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, embedding_size, num_encoders, hidden_size, bidirectional=False, cell='GRU',dropout=0.0):
        super(EncoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_encoders = num_encoders
        self.bidirectional = bidirectional
        self.cell = cell
        self.embedding_size = embedding_size
        self.embedding = nn.Embedding(input_size, embedding_size)
        self.dropout = nn.Dropout(dropout)

        #----------------------RNN----------------------------------------------
        if self.cell == 'RNN':
            self.encoder_rnn = nn.RNN(embedding_size, hidden_size, num_layers=num_encoders,
                                      bidirectional=bidirectional )
        
        #----------------------GRU----------------------------------------------
        elif self.cell == 'GRU':
            self.encoder_rnn = nn.GRU(embedding_size, hidden_size, num_layers=num_encoders,
                                      bidirectional=bidirectional)

        #----------------------LSTM---------------------------------------------
        else:
            self.encoder_rnn = nn.LSTM(embedding_size, hidden_size, num_layers=num_encoders,
                                       bidirectional=bidirectional )
            
    #-------------------------initialize hidden layer---------------------------

    def init_hidden(self, batch_size):
      
        #------------------------Number of directions---------------------------
        num_directions = 2 if self.bidirectional else 1

        #------------------------initialize Hidden Layer------------------------
        if self.cell == 'LSTM':
            return (torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device),
                    torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device))
        else:
          return torch.zeros(self.num_encoders * num_directions, batch_size, self.hidden_size, device=device)
        
    def forward(self, input_seq):
        batch_size = input_seq.size(1)
        
        # Embedding layer
        embedded = self.embedding(input_seq.long())
        
        embedded = self.dropout(embedded)
        # Encoder layers
        encoder_hidden = self.init_hidden(batch_size)

        # print(encoder_hidden.shape)
        if self.cell == 'LSTM':
            (encoder_hidden,encoder_cell) = self.init_hidden(batch_size)
            encoder_outputs, (encoder_hidden, encoder_cell) = self.encoder_rnn(embedded, (encoder_hidden, encoder_cell))
            encoder_hidden = (encoder_hidden,encoder_cell)
        else:
          
          encoder_outputs, encoder_hidden = self.encoder_rnn(embedded, encoder_hidden)
        
        
        return  encoder_outputs,encoder_hidden



class DecoderRNN(nn.Module):
    def __init__(self, output_size, embedding_size, num_encoders, num_decoders, hidden_size, dropout=0.0, cell='GRU'):
        super(DecoderRNN, self).__init__()
        
        self.hidden_size = hidden_size
        self.num_encoders = num_encoders
        self.num_decoders = num_decoders
        self.cell = cell
        self.embedding_size = embedding_size

        #------Initialize Dropout , decoder_fc(out),softmax function------------
        self.dropout = nn.Dropout(dropout)
        self.decoder_fc = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax(dim=-1)

        #-----------------------Initialize Embedding layer----------------------
        self.embedding = nn.Embedding(output_size, embedding_size)
        

        if self.cell == 'RNN':
            self.decoder_rnn = nn.RNN(embedding_size, hidden_size, num_layers=num_decoders)
        elif self.cell == 'GRU':
            self.decoder_rnn = nn.GRU(embedding_size, hidden_size, num_layers=num_decoders)
                                      
        else :
            self.decoder_rnn = nn.LSTM(embedding_size, hidden_size, num_layers=num_decoders)                      
                                     


    def forward(self, input_seq, hidden):
        batch_size = input_seq.shape[0]
        

        
        decoder_input = input_seq

        decoder_input = decoder_input.unsqueeze(1)
        
        # Embedding layer aaplying embedding and dropout

        embedded = self.embedding(decoder_input.long()).view(-1,batch_size,self.embedding_size)
        # embedded = self.dropout(embedded)
        decoder_hidden = hidden

        decoder_output , decoder_hidden = self.decoder_rnn(embedded, decoder_hidden)
       
        # Project the output to the output size and apply softmax

        decoder_output = self.softmax(self.decoder_fc(decoder_output))
        
        # Return the output tensor
        return decoder_output,decoder_hidden

## Training and Evalution Function for normal decoder

In [ ]:

#---------Train Seq2Seq Model--------------------------

def trainSeq2Seq(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epoch,learn_rate,teach_ratio):

  # input vocabaliry size
  input_size = data.input_lang.n_chars

  # output vocabaliry size
  output_size = data.output_lang.n_chars

  # Initialize Encoder , Decoder
  encoder = EncoderRNN( input_size, embedding_size, num_encoder, hidden_size, bidirectional, cell_type,dropout)
  decoder = DecoderRNN(output_size,embedding_size,num_encoder,num_decoder,hidden_size,dropout,cell_type)

  # Load Data into DataLoader
  trainData = TensorDataset(data.train_xE, data.train_yE)
  trainLoader = DataLoader(trainData,batch_size=batch_size,shuffle=True) 

  # Initialize Encoder Decoder optimizer and loss function
  encoder_optimizer=optim.Adam(encoder.parameters(),learn_rate)
  decoder_optimizer=optim.Adam(decoder.parameters(),learn_rate)
  loss_fun=nn.CrossEntropyLoss(reduction="sum")

  # Push encoder and decoder to device
  encoder.to(device)
  decoder.to(device)

  seq_len = 20
  for i in range(epoch):
  
    running_loss = 0.0
    train_correct = 0

    # Set Encoder and Decoder in Train mode
    encoder.train()
    decoder.train()

    # forward pass and backtrack for each batch

    for j,(train_x,train_y) in enumerate(trainLoader):
        loss=0

        seq_len = train_y.shape[1]

        # Load data to device
        train_x = train_x.to(device)
        train_y = train_y.to(device)
        
        x = train_x.T
        y = train_y.T

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # Encoder forward Pass
        encoder_output, encoder_hidden = encoder(x)
        

        # handle bidirectional case 
        if bidirectional:
          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2

        decoder_input =(y)[0]
        
        # handle layers mismatch
        decoder_hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)

        # pred_probabilities of batch
        pred_output=torch.zeros(y.shape[0],batch_size,output_size).to(device)
          
        
        for k in range(1,y.shape[0]):

          # Decoder Pass
          decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden)
          
          pred_output[k-1]=decoder_output

          # Get the index of the maximum probability in each row
          max_probs, indices = torch.max(decoder_output, dim=2)

          
          # Reshape the indices tensor to (batch_size,) for easier indexing
          indices = indices.view(batch_size)
          

          # teach forcing algorithm
          if((random.random()<teach_ratio)):
            decoder_input=(y)[k]
          else:
            decoder_input=indices.clone()

        # Decoder forward pass
        decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden)
          
        pred_output[-1]=decoder_output
        
        
        
        # Calculate max probalibity index and load it to device 
        preds = pred_output.argmax(dim=-1)
        preds = preds.T
        preds = preds.to(device)
        
        # Reshape tensors
        pred_output= pred_output.view(-1, pred_output.size(-1))  # Reshape to (batch_size * seq_len, vocab_size)
        y = y.reshape(-1)

        # calculate loss
        loss = loss_fun(pred_output, y.long())
        
        # add loss for each batch
        running_loss += loss.item()
        

        # Compute the gradients and update the parameters
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

       
        # Calculate correct words in each batch
        train_correct += count_exact_matches(preds,train_y)

    # calculate AVg_accuracy and avg_loss      
    avg_loss = running_loss / (batch_size * len(trainLoader)*seq_len)
    avg_acc = train_correct/(batch_size*len(trainLoader))

        
    #calculate validation accuracy and loss for each epoch and log all parameters to waandb
    val_loss,val_acc ,preds= evaluate(data.val_xE,data.val_yE,encoder, decoder, device,output_size,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type)
    wandb.log({'Train_loss': avg_loss, 'Train_acc': avg_acc*100,'Epoch':i+1,'Val_loss':val_loss,'Val_Acc':val_acc*100})
  return encoder ,decoder


#----------------------------------evaluate on data loss and accuracy-----------

def evaluate(X,Y,encoder, decoder, device,output_size,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type):

    # Set ENCODER and Decoder in evalution Mode
    encoder.eval()
    decoder.eval()

    #predictions for whole Dataset

    predicted = None
    running_loss =0
    correct=0
    total=0
    Data = TensorDataset(X,Y)
    Loader = DataLoader(Data,batch_size=batch_size,shuffle=False) 
    loss_fun=nn.CrossEntropyLoss(reduction="sum")
    with torch.no_grad():
      for j,(xx,yy) in enumerate(Loader):
        loss=0

        # Push batch to Device
        xx = xx.to(device)
        yy = yy.to(device)

        x = xx.transpose(0,1)
        y = yy.transpose(0,1)

        # Encoder forward pass
        encoder_output, encoder_hidden = encoder(x)
        
        # handle bidirectional case and LSTM cell type
        if bidirectional:
          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2

        # Initialize the decoder_hidden tensor
          
        
        
        decoder_input =(y)[0]
        # print("encoder_hidden:-",encoder_hidden.shape)
        decoder_hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)
        
        pred_output=torch.zeros(y.shape[0],batch_size,output_size)
          

        # Decoding for one batch
        for k in range(1,y.shape[0]):

          
          decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden)
          
          pred_output[k-1]=decoder_output

          # Get the index of the maximum probability in each row
          max_probs, indices = torch.max(decoder_output, dim=2)

          # Reshape the indices tensor to (batch_size,) for easier indexing
          indices = indices.view(batch_size)
          decoder_input=indices.clone()

        decoder_output,decoder_hidden= decoder(decoder_input, decoder_hidden) 
        pred_output[-1]=decoder_output

        preds = pred_output.argmax(dim=-1)
        preds = preds.T
        
        # Concat for whole dataSet
        if(predicted==None):
          predicted = preds
        else:
          predicted = torch.cat([predicted,preds],dim=0)
        
        # Reshape to (batch_size * seq_len, vocab_size) and Load to device
        pred_output= pred_output.view(-1, pred_output.size(-1))  
        y = y.reshape(-1)

        pred_output = pred_output.to(device)
        y =y.to(device)

        # Calculate  Loss
        loss = loss_fun(pred_output, y.long())
        running_loss += loss.item()
        
        # Calculate correct predictions
        correct += count_exact_matches(preds,yy)
        
    # Loss and Accuracy for Dataset    
    avg_loss = running_loss / (batch_size * (X.shape[0]/batch_size)*Y.shape[1])
    avg_acc = correct / X.shape[0]
    
    return avg_loss , avg_acc , predicted



## Supporting functions

In [ ]:

# -------------------Count Number of correct words------------------------------

def count_exact_matches(pred, target):
    """
    Counts the number of rows in preds tensor that match exactly with each row in y tensor.
    preds: tensor of shape (batch_size, seq_len)
    y: tensor of shape (batch_size, seq_len)
    """
    pred = pred[:, 1:-1] # ignore first and last index of each row
    target = target[:, 1:-1] # ignore first and last index of each row
    count=0;
    for i in range(pred.shape[0]):
      flag = True
      for j in range(target.shape[1]):
        if(target[i][j].item()!=0 and target[i][j].item()!=pred[i][j].item()):
          flag=False
          break;
         
      if(flag):
        # print("---------Correct-----")
        # print(pred[i])
        # print(target[i])
        count+=1;
    # print(count)
    return count


        

#------------------------------ Confusion Matrix creation----------------------- 

def confusion_matrix (predicted , target , output_dict_size):
  '''Create Confusion matrix of shape output_dict_size
     and return it'''
  CM = np.zeros((output_dict_size,output_dict_size))

  for i in range(target.shape[0]):
    for j in range(target.shape[1]):
       pred = int(predicted[i][j])
       targ = int(target[i][j])
       CM[pred][targ]+=1
  
  return CM



def plot_confusion_matrix(cm,vocab):


  # This code will plot Confusion matrix


  classes =[]

  for i in range(cm.shape[0]):
    classes.append(vocab[i])

  print(classes)
  # Calculate the percentages
  percentages = (cm / np.sum(cm)) * 100

  # Define the text for each cell
  cell_text = []
  for i in range(len(classes)):
      row_text = []
      for j in range(len(classes)):

          txt = "Total "+f'{cm[i, j]}Per. ({percentages[i, j]:.3f})'
          if(i==j):
            txt ="Correcty Predicted " +classes[i]+""+txt
          if(i!=j):
            txt ="Predicted " +classes[j]+" For "+classes[i]+""+txt
          row_text.append(txt)
      cell_text.append(row_text)

  # Define the trace
  trace = go.Heatmap(z=percentages,
                    x=classes,
                    y=classes,
                    colorscale='Blues',
                    colorbar=dict(title='Percentage'),
                    hovertemplate='%{text}%',
                    text=cell_text,
                    )

  # Define the layout
  layout = go.Layout(title='Confusion Matrix',
                    xaxis=dict(title='Predicted Character'),
                    yaxis=dict(title='True Character'),
                    )

  # Plot the figure
  fig = go.Figure(data=[trace], layout=layout)
  return fig
  #wandb.log({'confusion_matrix': (fig)})


# Generate Decoder hidden layer according to number of decoder and decoder and cell_type
 

def generateDecoderHidden(cell,num_encoder,num_decoder,encoder_hidden):
    hidden={}

    # ------------For LSTM cell Type--------------------

    if(cell=="LSTM"):
      
      hx,state = (encoder_hidden)

      # number of encoder > number of decoder

      if(num_encoder>num_decoder):
        hx = hx[num_encoder-num_decoder:]
        state = state[num_encoder-num_decoder:]
      
      # number of encoder < number of decoder

      elif(num_encoder<num_decoder):
        top = hx[-1].unsqueeze(0).clone()
        extra = num_decoder-num_encoder
        hiddenExtra =top.repeat(extra,1,1)
        hx =torch.cat((hx,hiddenExtra),dim=0)
      
        stop = state[-1].unsqueeze(0).clone()
        extra = num_decoder-num_encoder
        stateExtra =top.repeat(extra,1,1)
        state =torch.cat((state,stateExtra),dim=0)

      hidden=(hx,state)

    # -------------------For Other cell type--------

    else :

      hidden = encoder_hidden
      
      # number of encoder > number of decoder
      if(num_encoder>num_decoder):
        hidden = encoder_hidden[num_encoder-num_decoder:]

      # number of encoder < number of decoder 
      elif(num_encoder<num_decoder):
        top = encoder_hidden[-1].unsqueeze(0).clone()
        extra = num_decoder-num_encoder
        hiddenExtra =top.repeat(extra,1,1)
        hidden =torch.cat((encoder_hidden,hiddenExtra),dim=0)

    return hidden



# test_loss,test_acc,preds = evaluate(data.test_xE,data.test_yE,encoder, decoder, device,data.output_lang.n_chars,len(data.test_xE),bidirectional,hidden_size,num_encoder,num_decoder,cell_type)

# print(test_loss , test_acc)  


## Attention Decoder

In [ ]:
class AttentionDecoderRNN(nn.Module):
    def __init__(self, hidden_size, embedding_size , output_size, num_decoder, cell_type, dropout=0.0):
        super(AttentionDecoderRNN, self).__init__()

        # Define the input size, hidden size, output size, and number of layers
        self.embedding_size = embedding_size
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.num_layers = num_decoder
        self.cell_type = cell_type

        # Define the embedding layer
        self.embedding = nn.Embedding(output_size, embedding_size)
        self.fc = nn.Linear(hidden_size, output_size)

        # Define the attention mechanism
        self.attention = nn.Linear(hidden_size + embedding_size, hidden_size)
        self.context_vector = nn.Parameter(torch.zeros(hidden_size))
        self.energy = nn.Linear(hidden_size * 2, 1)
        self.softmax = nn.Softmax(dim=0)

        # Define the output layer and relu
        self.output_layer = nn.Linear(hidden_size, output_size)
        self.relu = nn.ReLU()

         # Define the dropout layer
        self.dropout = nn.Dropout(dropout)

        # Define softmax
        self.logsoftmax = nn.LogSoftmax(dim=-1)

        # Define the RNN layer
        if cell_type == "RNN":
            self.decoder_rnn = nn.RNN(embedding_size + hidden_size, hidden_size, num_layers=num_decoder)
        elif cell_type == "GRU":
            self.decoder_rnn = nn.GRU(embedding_size + hidden_size, hidden_size, num_layers=num_decoder)
        elif cell_type == "LSTM":
            self.decoder_rnn = nn.LSTM(embedding_size + hidden_size, hidden_size, num_layers=num_decoder)
        else:
            raise ValueError("Invalid cell type")

       

    def forward(self, input, hidden, encoder_outputs):

        # Convert to [1,batch_size] 
        input = input.unsqueeze(0)

        seq_len = encoder_outputs.shape[0]
        
        # Embedding layer and Dropout
        embedding = self.embedding(input.long())
        



        cell=None
        if(self.cell_type=="LSTM"):
          hidden ,cell= hidden 
       
         hidden_new= hidden.repeat(seq_len, 1,  1)  # repeat along the new dimension
        
        
        
        #-------------------------Attention Mechanism---------------------------
        energy = self.relu(self.energy(torch.cat((hidden_new, encoder_outputs), dim=2)))
        attention = self.softmax(energy)
        context_vector = torch.einsum("snk,snl->knl", attention, encoder_outputs)
        rnn_input = torch.cat((context_vector, embedding), dim=2)

        if(self.cell_type=="LSTM"):
          hidden = (hidden,cell)
        
        outputs, decoder_hidden= self.decoder_rnn(rnn_input, hidden)
       
        predictions = self.fc(outputs)
       
        output = self.logsoftmax(predictions)
        
        return output, decoder_hidden ,attention

## Trainining of Attention decoder and evalution for Attention

In [ ]:

def trainAttension(data,cell_type="RNN",embedding_size=64,num_encoder=2,num_decoder=2,hidden_size=32,batch_size=16,bidirectional=False,dropout=0.0,beam_width=0,epoch=10,learn_rate=0.001,teach_ratio=0.0):
  
  #input/output lang vocabalary size
  input_size = data.input_lang.n_chars
  output_size = data.output_lang.n_chars

  # initialize encoder and AttentionDecoder
  encoder = EncoderRNN( input_size, embedding_size, num_encoder, hidden_size, bidirectional, cell_type,dropout)
  decoder = AttentionDecoderRNN(cell_type=cell_type,dropout=dropout,embedding_size=embedding_size,hidden_size=hidden_size,num_decoder=num_decoder,output_size=output_size)

  # pushData to Loader with given batchSize
  trainData = TensorDataset(data.train_xE, data.train_yE)
  trainLoader = DataLoader(trainData,batch_size=batch_size,shuffle=True) 

  # Initialize optimizer and Loss Function
  encoder_optimizer=optim.Adam(encoder.parameters(),learn_rate)
  decoder_optimizer=optim.Adam(decoder.parameters(),learn_rate)
  loss_fun=nn.CrossEntropyLoss(reduction="sum")

  # push Encoder and decoder Device
  encoder.to(device)
  decoder.to(device)

  seq_len = 20
  for i in range(epoch):
  
    running_loss = 0.0
    correct = 0

    # Encoder , Decoder in Train Mode
    encoder.train()
    decoder.train()
    
    for j,(train_x,train_y) in enumerate(trainLoader):
        loss=0

        # Seq_len of output
        seq_len = train_y.shape[1]

        # push batch to device
        train_x = train_x.to(device)
        train_y = train_y.to(device)

        # transpose Batch dat
        x = train_x.transpose(0, 1)
        y = train_y.transpose(0, 1)

        encoder_optimizer.zero_grad()
        decoder_optimizer.zero_grad()

        # encoder forward pass
        encoder_output, encoder_hidden = encoder(x)
        

        # Handle birectional and LSTM case
        if bidirectional:
          hidden_size = encoder_output.size(2) // 2
          forward_output = encoder_output[:, :, :hidden_size]
          backward_output = encoder_output[:, :, hidden_size:]
          encoder_output = torch.add(forward_output, backward_output)/2

          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2

        # Initialize the decoder_hidden , decoder_input tensor
          
        
        
        decoder_input =(y)[0]
        
        decoder_hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)
        
        pred_output=torch.zeros(y.shape[0],batch_size,output_size).to(device)
          
        
        for k in range(1,y.shape[0]):

          
          decoder_output,decoder_hidden,attention= decoder(decoder_input, decoder_hidden,encoder_output)
          
          pred_output[k-1]=decoder_output

          # Get the index of the maximum probability in each row
          max_probs, indices = torch.max(decoder_output, dim=2)

         
          # Reshape the indices tensor to (batch_size,) for easier indexing
          indices = indices.view(batch_size)
          
          if((random.random()<teach_ratio)):
            decoder_input=(y)[k]
          else:
            decoder_input=indices.clone()

        decoder_output,decoder_hidden,attention= decoder(decoder_input, decoder_hidden,encoder_output)
        pred_output[-1]=decoder_output
        
        # get predicted indices (words for batch)
        preds = pred_output.argmax(dim=-1)
        preds = preds.T    #[batch_size , seq_len]
        

        pred_output= pred_output.view(-1, pred_output.size(-1))  # Reshape to (batch_size * seq_len, vocab_size)
        y = y.reshape(-1)

        
        # compute loss
        loss = loss_fun(pred_output, y.long())
        running_loss += loss.item()

        # Compute the gradients and update the parameters
        loss.backward()
        encoder_optimizer.step()
        decoder_optimizer.step()

        # Compute correct matches

        correct += count_exact_matches(preds,train_y)
       
    # loss and accuracy for whole data    
    avg_loss = running_loss / (batch_size * len(trainLoader)*seq_len)
    avg_acc = correct/(batch_size*(len(trainLoader)))
    

    running_loss = 0.0
    correct = 0
            
    val_loss,val_acc,preds = evaluateAttension(data.val_xE,data.val_yE,encoder, decoder, device,output_size,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type)
    wandb.log({'Train_loss': avg_loss, 'Train_acc': avg_acc*100,'Epoch':i+1,'Val_loss':val_loss,'Val_Acc':val_acc})
  return encoder ,decoder



def evaluateAttension(X,Y,encoder, decoder, device,output_size,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type):
    
    # Set encoder decoder in evalution Mode
    encoder.eval()
    decoder.eval()

    # Initialize predicted , runningloss , correct ( predictions)
    predicted=None
    running_loss =0
    correct=0
    
    # Load data into Loader and initialize loss function
    Data = TensorDataset(X,Y)
    Loader = DataLoader(Data,batch_size=batch_size,shuffle=False) 
    loss_fun=nn.CrossEntropyLoss(reduction="sum")

    # Calculate loss for whole data and total correct predictions 
    with torch.no_grad():
      for j,(xx,yy) in enumerate(Loader):
        
        #  push Batch to device
        x = xx.to(device)
        y = yy.to(device)

        x = x.transpose(0,1)
        y = y.transpose(0,1)

        # Encoder forawrd pass
        encoder_output, encoder_hidden = encoder(x)
        
        #handle bidirectional case
        if bidirectional:
          hidden_size = encoder_output.size(2) // 2
          forward_output = encoder_output[:, :, :hidden_size]
          backward_output = encoder_output[:, :, hidden_size:]
          encoder_output = torch.add(forward_output, backward_output)/2

          # print(hidden_size)
          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2

        # Initialize the decoder_hidden tensor and decoder_input
        decoder_input =(y)[0]
        decoder_hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)
        
        
        pred_output=torch.zeros(y.shape[0],batch_size,output_size).to(device)
        for k in range(1,y.shape[0]):

          
          decoder_output,decoder_hidden,attention= decoder(decoder_input, decoder_hidden,encoder_output)
          
          pred_output[k-1]=decoder_output

          # Get the index of the maximum probability in each row
          max_probs, indices = torch.max(decoder_output, dim=2)

          
          # Reshape the indices tensor to (batch_size,) for easier indexing
          indices = indices.view(batch_size)
          decoder_input=indices.clone()

        decoder_output,decoder_hidden,attention= decoder(decoder_input, decoder_hidden,encoder_output) 
        pred_output[-1]=decoder_output
       
        
        # Get predicted Words
        preds = pred_output.argmax(dim=-1)
        preds = preds.T

        if(predicted==None):
          predicted = preds
        else:
          predicted = torch.cat([predicted,preds],dim=0)
        preds=preds.to(device)

        # Reshape to (batch_size * seq_len, vocab_size)
        pred_output= pred_output.view(-1, pred_output.size(-1))  
        y = y.reshape(-1)
        
        # calculate loss and correct predictions for batch
        loss = loss_fun(pred_output, y.long())
        running_loss += loss.item()
        correct += count_exact_matches(preds,yy)
          
        
    avg_loss = running_loss / (batch_size * (X.shape[0]/batch_size)*Y.shape[1])
  
    avg_acc = 100*correct / X.shape[0]
    
    return avg_loss , avg_acc , predicted
    
# encoder,decoder=trainAttension(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epoch,learn_rate,teach_ratio)

  


## Import Data for language 


In [ ]:

data = EncodedData("hin")    


## Wandb configuration for sweep runs


In [ ]:
def wandb_runs(data):

      #-------Wandb configurations--------

    config = {
        "project":"CS6910_Assignment3_",
        "method": 'bayes',
        "metric": {
        'name': 'Val_Acc',
        'goal': 'maximize'
        },
        'parameters' :{
        "epoch": {"values":[30]},
        "learn_rate": {"values":[0.001]},
        "batch_size": {"values": [256,64,32]},
        "embedding_size": {"values":[1024,512,256,64]},
        "hidden_size": {"values":[ 1024,512,256,64]},
        "encoder_layers": {"values":[1]},
        "decoder_layers": {"values":[1]},
        "cell_type": {"values":["RNN"]},
        "bi_directional":{"values":["No"]},
        "dropout":{"values":[0.0,0.2,0.3]},
        "attention":{"values":["Yes"]},
        "teach_ratio":{"values":[0.0,0.2,0.4,0.6]}
        
        }
    }

    #------Train function-----------

    def train_rnn():

      #-----initialize project---------

      
        wandb.init()
        
       
        # ---- Sweep Name-----

        name='_CT_'+str(wandb.config.cell_type)+"_BS_"+str(wandb.config.batch_size)+"_EPOCH_"+str(wandb.config.epoch)+"_ES_"+str(wandb.config.embedding_size)+"_HS_"+str(wandb.config.hidden_size)
        wandb.run.name = name
        

        # parameters to pass to functions
        
        learn_rate = wandb.config.learn_rate
        batch_size = wandb.config.batch_size
        hidden_size = wandb.config.hidden_size
        embedding_size = wandb.config.embedding_size
        num_encoder = wandb.config.encoder_layers
        num_decoder = wandb.config.decoder_layers
        cell_type = wandb.config.cell_type
        bidirectional = (wandb.config.bi_directional=="Yes")
        dropout = wandb.config.dropout
        teach_ratio = wandb.config.teach_ratio
        epoch = wandb.config.epoch
        beam_width =0
        encoder =None
        decoder = None


        if wandb.config.attention == "Yes" :
            
            encoder,decoder=trainAttension(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epoch,learn_rate,teach_ratio)
        else:
            
            encoder,decoder=trainSeq2Seq(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epoch,learn_rate,teach_ratio)
        
        wandb.run.save()
        wandb.run.finish()
    sweep_id=wandb.sweep(config,project="CS6910_Assignment3__AAtention")
    

    # ------Number of experiments we want to run--------
    
    wandb.agent(sweep_id,function=train_rnn,count=50)
#     wandb.agent(sweep_id,function=train_rnn,count=10)
#     wandb.agent(sweep_id,function=train_rnn,count=10)
#     wandb.agent(sweep_id,function=train_rnn,count=10)
#     wandb.agent(sweep_id,function=train_rnn,count=10)
#     wandb.agent(sweep_id,function=train_rnn,count=10)
#     wandb.agent(sweep_id,function=train_rnn,count=10)
#     wandb.agent(sweep_id,function=train_rnn,count=10)
#     wandb.agent(sweep_id,function=train_rnn,count=10)


## Wandb sweep runs


In [ ]:

wandb.finish()
wandb_runs(data)

## Translate prediction to proper words according to dictionary  and save in DataFrame formate (Input , predicted,Actual)


In [ ]:
def translate_prediction(InDict , input,OutDict,pred,target):
    
    '''give input and pred in shape of batch_size*seq_len'''

    pred = pred[:, 1:-1] # ignore first and last index of each row
    input = input[:, :-1] # ignore  last index of each row
    target = target[:, 1:-1] # ignore  last index of each row
    predictions = [] 
    Input = [] 
    Target = []
    for i in range(pred.shape[0]):
        
        stringP="" # Predicted word
        stringIn="" # Input word
        stringActu = "" # Actual string

        for j in range(pred.shape[1]):

            # Ignore padding
            if(target[i][j].item()!=0):
              
              stringP = stringP + OutDict[pred[i][j].item()]
              stringActu = stringActu+OutDict[target[i][j].item()]
                    
        for j in range(input.shape[1]):
            
               if(input[i][j].item()!=0):
                    
                    stringIn = stringIn + InDict[input[i][j].item()]   

        # Append words in respective List
        
        predictions.append(stringP)
        Input.append(stringIn)         
        Target.append(stringActu)   

    # Create a DataFrame
    df = pd.DataFrame({"input": Input, "predicted": predictions,"Actual":Target})

    return df

            

## Attention weight heatmap

In [ ]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np
def Attension_heatmap(X,Y,encoder, decoder, device,output_size,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type):
    
    # Assuming you have a tensor named 'data' with shape [num_samples, ...]
    num_samples = X.shape[0]
    num_selected_samples = batch_size

    # Generate random permutation of indices
    random_indices = torch.randperm(num_samples)[:num_selected_samples]

    # Select the corresponding samples from the tensor
    X = X[random_indices]
    Y = Y[random_indices]

    # Set encoder decoder in evalution Mode
    encoder.eval()
    decoder.eval()

    # Initialize predicted , runningloss , correct ( predictions)
    attention_weights=None
    
    # Load data into Loader and initialize loss function
   
    print(X.shape)
    # Calculate loss for whole data and total correct predictions 
    with torch.no_grad():
      
        
        #  push Batch to device
        x = X.to(device)
        y = Y.to(device)

        x = x.transpose(0,1)
        y = y.transpose(0,1)

        # Encoder forawrd pass
        encoder_output, encoder_hidden = encoder(x)
        
        #handle bidirectional case
        if bidirectional:
          hidden_size = encoder_output.size(2) // 2
          forward_output = encoder_output[:, :, :hidden_size]
          backward_output = encoder_output[:, :, hidden_size:]
          encoder_output = torch.add(forward_output, backward_output)/2

          # print(hidden_size)
          if(cell_type=="LSTM"):
            hidden,state = encoder_hidden
            hidden = hidden.resize(2,num_encoder,batch_size,hidden_size)
            hidden=torch.add(hidden[0],hidden[1])/2

            state = state.resize(2,num_encoder,batch_size,hidden_size)
            state=torch.add(state[0],state[1])/2
            encoder_hidden =(hidden,state)

          else:
            hidden=encoder_hidden.resize(2,num_encoder,batch_size,hidden_size)
            encoder_hidden=torch.add(hidden[0],hidden[1])/2

        # Initialize the decoder_hidden tensor and decoder_input
        decoder_input =(y)[0]
        decoder_hidden = generateDecoderHidden(cell_type,num_encoder,num_decoder,encoder_hidden)
        
        
        pred_output=torch.zeros(y.shape[0],batch_size,output_size).to(device)
        for k in range(1,y.shape[0]):

          
          decoder_output,decoder_hidden,attention= decoder(decoder_input, decoder_hidden,encoder_output)
          
        
          print(attention.shape)
          if(attention_weights==None):
            attention_weights=attention
          else :
            attention_weights = torch.cat([attention_weights,attention],dim = -1)

          # Get the index of the maximum probability in each row
          max_probs, indices = torch.max(decoder_output, dim=2)

          
          # Reshape the indices tensor to (batch_size,) for easier indexing
          indices = indices.view(batch_size)
          decoder_input=indices.clone()

        decoder_output,decoder_hidden,attention= decoder(decoder_input, decoder_hidden,encoder_output) 
        attention_weights = torch.cat([attention_weights,attention],dim = -1)
       
        print(attention_weights.shape)
        # Get predicted Words
        attention_weights = (attention_weights.cpu()).numpy()
        attention_weights = attention_weights.transpose(1, 2, 0)

        # Normalize the attention weights across the input sequence axis
        attention_weights = attention_weights / np.sum(attention_weights, axis=2, keepdims=True)
        print(attention_weights.shape)

        attention_weights_fixed = np.nan_to_num(attention_weights, nan=0.0000001)

        # Create a 3x3 grid of subplots for heatmaps
        fig = make_subplots(rows=4, cols=4)

        # Add heatmaps to the grid
        for i in range(batch_size):
            heatmap = attention_weights[i]
            
            x_label , y_label = [] , []

            for k in range(X.shape[1]):
              char = data.input_lang.index2char[(X[i][k]).item()]
              x_label.append(char)
            
            for k in range(Y.shape[1]):
              char = data.output_lang.index2char[(Y[i][k]).item()]
              y_label.append(char)
            # Create a heatmap trace
            heatmap_trace = go.Heatmap(z=heatmap,
                                      x=y_label,
                                      y=x_label,
                                      colorscale='hot',
                                      reversescale=True,
                                      showscale=False)

            # Add the heatmap trace to the subplot
            row = (i // 4) + 1
            col = (i % 4) + 1
            fig.add_trace(heatmap_trace, row=row, col=col)

           # Set subplot titles
            # Set subplot titles
    
            fig.update_xaxes(title_text="Output Sequence", row=row, col=col)
            fig.update_yaxes(title_text="Input Sequnece", row=row, col=col)

            # Update subplot titles
            fig.update_layout(title=f"Attention Heatmap Grid", height=800, width=800)

        # Show the grid of heatmaps
        wandb.log({"heatmap ":fig})

        # # Create a 3x3 grid of subplots for heatmaps
        # fig, axes = plt.subplots(3, 3, figsize=(12, 12))

        # # Plot the attention heatmaps in the grid
        # for i, ax in enumerate(axes.flatten()):
        #     heatmap = attention_weights[i]
        #     ax.imshow(heatmap, cmap='hot', interpolation='nearest')
        #     x_label , y_label = [] , []

        #     for k in range(X.shape[1]):
        #       char = data.input_lang.index2char[(X[i][k]).item()]
        #       x_label.append(char)
            
        #     for k in range(Y.shape[1]):
        #       char = data.output_lang.index2char[(Y[i][k]).item()]
        #       y_label.append(char)
        #     ax.set_xticks(np.arange(Y.shape[1]))
        #     ax.set_yticks(np.arange(X.shape[1]))
        #     ax.set_xticklabels(y_label)
        #     ax.set_yticklabels(x_label)

        #     ax.set_title(f'Sample {random_indices[i]}')
        #     ax.set_xlabel('Decoder Sequence ')
        #     ax.set_ylabel('Encoder Sequence ')

        # # Adjust spacing
        # plt.tight_layout()

        # # Log the figure to wandb
        # wandb.log({"attention_heatmaps": wandb.Image(fig)})

        # # Display the figure
        # plt.show()

        
    
    
   


## For Best run calculate test accuracy and get word to word prediction

In [ ]:
def Best_Run(data):
    
    config = {
        "project":"CS6910_Assignment3",
        "method": 'bayes',
        "metric": {
        'name': 'Val_acc',
        'goal': 'maximize'
        },
        'parameters' :{
        "epoch": {"values":[30]},
        "learn_rate": {"values":[0.001]},
        "batch_size": {"values": [256]},
        "embedding_size": {"values":[ 256]},
        "hidden_size": {"values":[1024]},
        "encoder_layers": {"values":[1]},
        "decoder_layers": {"values":[1]},
        "cell_type": {"values":["LSTM"]},
        "bi_directional":{"values":["Yes"]},
        "dropout":{"values":[0.2]},
        "attention":{"values":["No"]},
        "teach_ratio":{"values":[0.6]}
        
        }
    }
    def train_rnn():
        wandb.init()
        
       

        name='_CT_'+str(wandb.config.cell_type)+"_BS_"+str(wandb.config.batch_size)+"_EPOCH_"+str(wandb.config.epoch)+"_ES_"+str(wandb.config.embedding_size)+"_HS_"+str(wandb.config.hidden_size)
        wandb.run.name = name
        

        
        
        learn_rate = wandb.config.learn_rate
        batch_size = wandb.config.batch_size
        hidden_size = wandb.config.hidden_size
        embedding_size = wandb.config.embedding_size
        num_encoder = wandb.config.encoder_layers
        num_decoder = wandb.config.decoder_layers
        cell_type = wandb.config.cell_type
        bidirectional = (wandb.config.bi_directional=="Yes")
        dropout = wandb.config.dropout
        teach_ratio = wandb.config.teach_ratio
        epoch = wandb.config.epoch
        beam_width =0
        encoder =None
        decoder = None
        test_loss = None
        test_acc = None
        preds = None

        if wandb.config.attention == "Yes" :
            
            encoder,decoder=trainAttension(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epoch,learn_rate,teach_ratio)
            test_loss,test_acc,preds = evaluateAttension(data.test_xE,data.test_yE,encoder, decoder, device,data.output_lang.n_chars,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type)
            wandb.log({"test_loss":test_loss,"test_acc":test_acc})
            Attension_heatmap(data.test_xE,data.test_yE,encoder,decoder,device,data.output_lang.n_chars,16,bidirectional,hidden_size,num_encoder,num_decoder,cell_type)
            dataframe = translate_prediction(data.input_lang.index2char,data.test_xE,data.output_lang.index2char,preds,data.test_yE)
            dataframe.to_csv("predictions_Attention.csv")
        else:
            
            encoder,decoder=trainSeq2Seq(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epoch,learn_rate,teach_ratio)
            test_loss,test_acc,preds = evaluate(data.test_xE,data.test_yE,encoder, decoder, device,data.output_lang.n_chars,batch_size,bidirectional,hidden_size,num_encoder,num_decoder,cell_type)
            wandb.log({"test_loss":test_loss,"test_acc":test_acc*100})
            dataframe = translate_prediction(data.input_lang.index2char,data.test_xE,data.output_lang.index2char,preds,data.test_yE)
            dataframe.to_csv("predictions_Vannila.csv")
        
            
            

        
        
        
        

        print("confusion matrix start")
        CM = confusion_matrix(preds,data.test_yE,data.output_lang.n_chars)

        print("confusion matrix done")
        fig = plot_confusion_matrix(CM,data.output_lang.index2char)
        print("fig done")
        wandb.log({'confusion_matrix': (fig)})

        wandb.run.save()
        wandb.run.finish()
    sweep_id=wandb.sweep(config,project="CS6910_Assignment3_Best_run")
    
    
    wandb.agent(sweep_id,function=train_rnn,count=1)
# wandb.finish()  
# Best_Run(data)

In [ ]:


# learn_rate = 0.001
# batch_size = 32
# hidden_size = 256
# embedding_size = 256
# num_encoder = 2
# num_decoder = 2
# cell_type = 'LSTM'
# bidirectional =True
# dropout = 0.2
# teach_ratio = 0.5
# epoch = 30
# beam_width =0
# encoder,decoder=trainSeq2Seq(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epoch,learn_rate,teach_ratio)

  


In [ ]:
attention= "No"
learn_rate = 0.001
batch_size = 128
hidden_size = 512
embedding_size = 512
num_encoder = 3
num_decoder = 4
cell_type = 'GRU'
bidirectional =True
dropout = 0.5
teach_ratio = 0.5
epoch = 20
beam_width =0

In [ ]:
# encoder = None
# decoder = None
# wandb.init()
# if attention == "Yes" :
            
#     encoder,decoder=trainAttension(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epoch,learn_rate,teach_ratio)
# else:

#     encoder,decoder=trainSeq2Seq(data,cell_type,embedding_size,num_encoder,num_decoder,hidden_size,batch_size,bidirectional,dropout,beam_width,epoch,learn_rate,teach_ratio)


In [ ]:
 
        
# wandb.log({"test_loss":test_loss,"test_acc":test_acc})


In [ ]:
# preds = preds.to(device)

# dataframe = translate_prediction(data.input_lang.index2char,data.test_xE,data.output_lang.index2char,preds,data.test_yE)
# dataframe.to_csv("predictions.csv")